In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext line_profiler

ModuleNotFoundError: No module named 'line_profiler'

In [40]:
import pandas as pd
import numpy as np
import pulp
import random
import matplotlib.pyplot as plt
import sys
import os
#sys.path.append(os.path.abspath("C:\\Users\\Christophe GOUDET\\Documents\\Programmes\\RoomOptimisation"))
from LibRoomOptimisation import CreatePersoData, CreateOfficeData

In [41]:
persoData = pd.read_csv('persoData.csv')

In [42]:
#persoData=persoData[persoData['isCodir']==0]
#persoData=persoData[persoData['isGRC']==0]
persoData.head(13)

,Unnamed: 0,Nom,rawPerso1,rawPerso2,rawPerso3,rawEtage,rawWindow,rawClim,rawSeul,rawPassage,...,weightPerso3,perso3,inService,isCodir,secure,isGRC,isFace,isAgathe,mur,phone
0,0,Irwin L.,NaN,NaN,NaN,5,2.0,3.0,4.0,1.0,...,1,NaN,SI,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,Jean-Baptiste S.,NaN,NaN,NaN,4,1.0,1.0,4.0,1.0,...,1,NaN,SI,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,Romain V.,Marie Charlotte D.,Marine M.,Judith D.,5,2.0,4.0,3.0,1.0,...,1,Judith D.,SI,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,Sylvain Q.,Charlotte D.,NaN,NaN,3,4.0,2.0,5.0,1.0,...,1,NaN,OPS,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,4,Zineb F.,NaN,NaN,NaN,2,5.0,1.0,4.0,3.0,...,1,NaN,Achat,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,Elise P.,Antoine T.,Judith D.,Vincent L.,5,1.0,2.0,2.0,3.0,...,1,Vincent L.,BizDev,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,6,Jennifer P.,Emilie L.,Mathilde B.,Amélie M.,4,5.0,2.0,3.0,1.0,...,1,Amélie M.,SI,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,7,Marie Charlotte D.,Romain V.,Marine M.,Olivier B.,1,2.0,5.0,4.0,3.0,...,1,Olivier B.,Finance,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,8,Pauline A.,Agathe G.,Judith D.,Amélie M.,1,5.0,3.0,2.0,4.0,...,1,Amélie M.,MPC,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,9,Jean-Christophe L.,Charlotte D.,Jennifer P.,Sarah D.,4,2.0,3.0,3.0,2.0,...,1,Sarah D.,Finance,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [43]:
persoFilter = pd.DataFrame()
for x in range(1,4) :
    table = pd.pivot_table( persoData.loc[:,['weightPerso'+ str(x),'perso'+ str(x)]], values='weightPerso'+ str(x), columns=['perso' + str(x)], index=persoData.index, aggfunc='sum').fillna(0)
    persoFilter = persoFilter.add(table, fill_value=0).fillna(0)
    print(persoFilter)
persoFilter.sum().sort_values(ascending=False)


perso1  Agathe G.  Antoine T.  Charlotte D.  Elise P.  Emilie L.  \
2             0.0         0.0           0.0       0.0        0.0   
3             0.0         0.0           6.0       0.0        0.0   
5             0.0         6.0           0.0       0.0        0.0   
6             0.0         0.0           0.0       0.0        6.0   
7             0.0         0.0           0.0       0.0        0.0   
8             6.0         0.0           0.0       0.0        0.0   
9             0.0         0.0           6.0       0.0        0.0   
11            0.0         0.0           0.0       0.0        0.0   
12            0.0         0.0           6.0       0.0        0.0   
13            0.0         6.0           0.0       0.0        0.0   
14            0.0         0.0           0.0       0.0        0.0   
15            0.0         0.0           0.0       0.0        0.0   
16            0.0         0.0           0.0       0.0        0.0   
17            0.0         0.0           0.0     

Antoine T.            35.0
Charlotte D.          30.0
Romain V.             25.0
Pauline A.            22.0
Vincent L.            18.0
Agathe G.             14.0
Jennifer P.           13.0
Marine M.             12.0
Marie Charlotte D.    12.0
Savinien F.           10.0
Jean-Baptiste S.      10.0
Amélie M.              8.0
Olivier B.             8.0
Judith D.              7.0
Olivier A.             6.0
Emilie L.              6.0
Elise P.               6.0
Romain R.              6.0
Sarah D.               5.0
Laetitia F.            3.0
Mathilde B.            3.0
Florie W.              3.0
Christophe G.          3.0
Alexandra B.           3.0
Irwin L.               1.0
Dounia R.              1.0
Charlotte De Poix      1.0
Sylvain Q.             1.0
Zineb F.               1.0
dtype: float64